In [ ]:
"""
SIFT_Analysis_R3.0.py: EEG Analysis Toolbox for Epilepsy and Parkinson's Research
- Offline analysis of CHB-MIT EEG data (chb01_03.edf for seizures, chb01_01.edf for normal).
- Extracts features (spikes, band powers, PLV) using wavelet denoising and circle state processing.
- Outputs: Processed EEG, feature arrays, PSD plots, logs.
- Dependencies: mne, numpy, scipy, matplotlib, pywt.
- License: GNU GPL v3.0
- Ethical Statement: For research only, no therapeutic use.
"""

import os
import logging
import numpy as np
import mne
import scipy.signal as signal
import matplotlib.pyplot as plt
import pywt

# Setup directories and logging
eeg_data_dir = '/mnt/datatank/eeg_data/'
output_dir = './output_sift_analysis/'
os.makedirs(output_dir, exist_ok=True)
logging.basicConfig(filename=os.path.join(output_dir, 'sift_analysis.log'), level=logging.INFO)

# Parameters
fs = 256  # Sampling frequency (Hz)
channel = 'CZ-PZ'
kappa = 1e-2  # Circle state parameter
gamma = 1e-6
nu_density = 1e4
dt = 1 / fs
damping = 0.9999
seizure_start = 2996.0  # Seizure onset (seconds)

# Load EEG data
try:
    raw_seizure = mne.io.read_raw_edf(eeg_data_dir + 'chb01_03.edf', preload=True)
    raw_normal = mne.io.read_raw_edf(eeg_data_dir + 'chb01_01.edf', preload=True)
except FileNotFoundError as e:
    raise FileNotFoundError(f"Could not find {e.filename}. Ensure CHB-MIT dataset is in {eeg_data_dir}.")

raw_seizure.pick([channel])
raw_normal.pick([channel])
eeg_seizure = raw_seizure.get_data()[0] * 1e6  # μV
eeg_normal = raw_normal.get_data()[0] * 1e6
t = np.arange(len(eeg_seizure)) / fs

# Simulate Parkinson's
beta_signal = 50 * np.sin(2 * np.pi * 20 * t)
eeg_parkinsons = eeg_normal + beta_signal

def wavelet_denoise(eeg_signal):
    """Wavelet denoising."""
    wavelet = 'sym8'
    level = 5
    coeffs = pywt.wavedec(eeg_signal, wavelet, level=level)
    sigma = np.median(np.abs(coeffs[-1])) / 0.6745
    thresh = sigma * np.sqrt(2 * np.log(len(eeg_signal)))
    coeffs[1:] = [pywt.threshold(c, thresh, mode='soft') for c in coeffs[1:]]
    denoised = pywt.waverec(coeffs, wavelet)[:len(eeg_signal)]
    return denoised

def circle_state_process(eeg_signal):
    """Circle state processing inspired by GUFE."""
    n = len(eeg_signal)
    psi = np.zeros(n, dtype=complex)
    psi[:n] = eeg_signal
    for i in range(1, n-1):
        laplacian = (psi[i+1] - 2*psi[i] + psi[i-1]) / dt**2
        d2psi_dt2 = kappa * laplacian + gamma * nu_density * psi[i]
        psi[i+1] = 2*psi[i] - psi[i-1] + dt**2 * d2psi_dt2
        psi[i+1] *= damping
    psi = np.real(psi)
    if np.var(psi) > 0:
        psi *= np.sqrt(np.var(eeg_signal) / np.var(psi))
    return psi

def compute_plv(signal1, signal2, fs):
    """Phase-locking value (PLV) inspired by Mucking_around1.py."""
    analytic1 = signal.hilbert(signal1)
    analytic2 = signal.hilbert(signal2)
    phase1 = np.angle(analytic1)
    phase2 = np.angle(analytic2)
    plv = np.abs(np.mean(np.exp(1j * (phase1 - phase2))))
    return plv

def extract_features(eeg_signal, fs, label, seizure_start=2996.0):
    """Extract features for analysis."""
    denoised = wavelet_denoise(eeg_signal)
    processed = circle_state_process(denoised)

    # Bandpass filters
    b_theta, a_theta = signal.butter(4, [4 / (fs / 2), 7 / (fs / 2)], btype='band')
    b_alpha, a_alpha = signal.butter(4, [8 / (fs / 2), 13 / (fs / 2)], btype='band')
    b_beta, a_beta = signal.butter(4, [13 / (fs / 2), 30 / (fs / 2)], btype='band')
    theta_signal = signal.filtfilt(b_theta, a_theta, processed)
    alpha_signal = signal.filtfilt(b_alpha, a_alpha, processed)
    beta_signal = signal.filtfilt(b_beta, a_beta, processed)

    # Spike and beta detection
    spike_threshold = 100  # μV
    beta_threshold = 30  # μV
    spikes = np.abs(processed) > spike_threshold
    betas = np.abs(beta_signal) > beta_threshold
    spike_count = np.sum(spikes)
    beta_count = np.sum(betas)

    # PSD
    freqs, psd = signal.welch(processed, fs, nperseg=fs*2)
    theta_power = np.sum(psd[(freqs >= 4) & (freqs <= 7)])
    alpha_power = np.sum(psd[(freqs >= 8) & (freqs <= 13)])
    beta_power = np.sum(psd[(freqs >= 13) & (freqs <= 30)])

    # PLV
    plv = compute_plv(processed, alpha_signal, fs)

    return [spike_count / len(processed), theta_power, alpha_power, beta_power, plv], processed

# Process data
features_e, processed_e = extract_features(eeg_seizure, fs, 'seizure')
features_n, processed_n = extract_features(eeg_normal, fs, 'normal')
features_p, processed_p = extract_features(eeg_parkinsons, fs, 'parkinsons')

# Logging
logging.info(f"Epilepsy Features: Spike Rate={features_e[0]:.2e}, Theta={features_e[1]:.2f}, Alpha={features_e[2]:.2f}, Beta={features_e[3]:.2f}, PLV={features_e[4]:.2f}")
logging.info(f"Normal Features: Spike Rate={features_n[0]:.2e}, Theta={features_n[1]:.2f}, Alpha={features_n[2]:.2f}, Beta={features_n[3]:.2f}, PLV={features_n[4]:.2f}")
logging.info(f"Parkinson's Features: Spike Rate={features_p[0]:.2e}, Theta={features_p[1]:.2f}, Alpha={features_p[2]:.2f}, Beta={features_p[3]:.2f}, PLV={features_p[4]:.2f}")

# Plotting
plt.figure(figsize=(10, 6))
start_idx = int(seizure_start * fs)
end_idx = start_idx + int(2 * fs)
plt.plot(t[start_idx:end_idx], eeg_seizure[start_idx:end_idx], label='Raw EEG', alpha=0.7)
plt.plot(t[start_idx:end_idx], processed_e[start_idx:end_idx], label='Processed EEG', alpha=0.9)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (μV)')
plt.title('SIFT EEG Analysis - Epilepsy (Seizure Onset)')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'sift_eeg_epilepsy.png'))
plt.close()

plt.figure(figsize=(10, 6))
plt.plot(t[:int(2 * fs)], eeg_parkinsons[:int(2 * fs)], label='Raw EEG', alpha=0.7)
plt.plot(t[:int(2 * fs)], processed_p[:int(2 * fs)], label='Processed EEG', alpha=0.9)
plt.xlabel('Time (s)')
plt.ylabel('Amplitude (μV)')
plt.title('SIFT EEG Analysis - Parkinson’s')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(output_dir, 'sift_eeg_parkinsons.png'))
plt.close()

# Save data
np.save(os.path.join(output_dir, 'features_epilepsy.npy'), features_e)
np.save(os.path.join(output_dir, 'features_normal.npy'), features_n)
np.save(os.path.join(output_dir, 'features_parkinsons.npy'), features_p)
np.save(os.path.join(output_dir, 'processed_epilepsy.npy'), processed_e)
np.save(os.path.join(output_dir, 'processed_parkinsons.npy'), processed_p)
np.save(os.path.join(output_dir, 'processed_normal.npy'), processed_n)

print(f"SIFT analysis complete. Results saved to {output_dir}")